### 10. Define Your Own Task
***
Brainstorm with your group, then define and implement a new interesting task based on the dataset at hand. We recommend you to download and use data of other time periods. The duration of this data should be at least three months. For example you may use January 2020 to March 2020 in which the major changes happened to daily’s commutes. This part is open-ended and you’ll be graded based on your creativity. You may also wish to go beyond, and use any other navigation- / traffic- related dataset and define your favorite task. This part is worth 20% credit of the project. You may discuss your ideas with the TA in designated office hours.

> Task definition: Our task is to predict the travelling time of roads from Los Angeles area1. The duration of this part is Q1 of 2020. We apply the same method to compute the travelling time and the coordinate as the part 2. To make prediction, we will train a tree-based regressor on several node and graph features. Note that we split 50% of the roads to be the training data, and leave the rest 50% of them to evaluate the model performance. Since it is a regression task, we use root of mean square error (RMSE) as our evaluation metric.

### load package
***

In [1]:
import json
import random

import igraph as ig
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestRegressor
from tqdm.auto import tqdm

### load data
***
The preprocessing is the same as that of part 2.

In [2]:
df = pd.read_csv("los_angeles-censustracts-2020-1-All-MonthlyAggregate.csv")
df = df[["sourceid", "dstid", "mean_travel_time"]].reset_index(drop=True)

df["source_id"] = df[["sourceid", "dstid"]].min(axis=1)
df["dst_id"] = df[["sourceid", "dstid"]].max(axis=1)
graph = df.groupby(by=["source_id", "dst_id"], as_index=False)["mean_travel_time"].mean()

with open("graph_data_part3.txt", "w") as f:
    for _, (source_id, dst_id, mean_travel_time) in enumerate(graph.values):
        f.write("{} {} {}\n".format(int(source_id), int(dst_id), mean_travel_time))

g = ig.Graph.Read(f="graph_data_part3.txt", format="ncol", directed=False)
gcc = g.components().giant()
gcc = gcc.to_networkx()

print("Number of nodes: ", len(gcc.nodes))
print("Number of edges: ", len(gcc.edges))

Number of nodes:  2651
Number of edges:  978316


In [3]:
# load location data
loc_data = {}
with open("los_angeles_censustracts.json", "r") as f:
    cur = json.loads(f.readline())
    for feature in cur["features"]:
        # sync with TA's helper code
        a = feature['geometry']['coordinates'][0]
        mean_coords = np.array(a if type(a[0][0]) == float else a[0]).mean(axis=0)
        loc_data[feature["properties"]["MOVEMENT_ID"]] = {
            "address": feature["properties"]["DISPLAY_NAME"], "mean_coords": mean_coords
        }

### prepare training data
***
We use the mean travelling time of roads as our prediction target.

In [4]:
def get_distance_by_index(i, j):
    coords1 = loc_data[gcc.nodes[i]["name"]]["mean_coords"]
    coords2 = loc_data[gcc.nodes[j]["name"]]["mean_coords"]
    return (((coords1 - coords2) ** 2).sum()) ** 0.5


edges = list(gcc.edges)
distances = [get_distance_by_index(s, t) for s, t in tqdm(edges)]
edge2time = nx.get_edge_attributes(gcc, "weight")
travel_times = [edge2time[e] for e in list(gcc.edges)]

# split train and test set
train_rate = 0.5
n_train = round(len(travel_times) * train_rate)
all_indices = list(range(len(travel_times)))
random.shuffle(all_indices)
train_indices = all_indices[:n_train]
test_indices = all_indices[n_train:]

# target: travelling time
travel_times = np.array(travel_times)
y_train = travel_times[train_indices]
y_test = travel_times[test_indices]

  0%|          | 0/978316 [00:00<?, ?it/s]

### Random predictor
***
To set a lower bound of the performance, we implement a simple random predictor that randomly predicts the travelling time from a normal distribution using the mean and standard deviation from the training set.

In [5]:
def compute_rmse(y, y_pred):
    return ((y - y_pred) ** 2).mean() ** 0.5


mu = y_train.mean()
std = y_train.std()
y_pred = np.random.normal(mu, std, len(y_test))
rmse = round(compute_rmse(y_test, y_pred), 3)
print(f"Test rmse: {rmse}")

Test rmse: 899.964


### Baseline: Only use edge length
***
If the traffic condition is good, the travelling time should only depend on the length of the road. As can be seen in the below cell, the performance is much better than the random predictor. It means that the length of road is surely an important factor of the travelling time. Next, we set this as our baseline and try to find if there are any other factors affecting the travelling time.

In [6]:
def fit_and_evaluate(X_train, y_train, X_test, y_test):
    print("Start fitting")
    print(f"X_train shape: {X_train.shape}; X_test shape: {X_test.shape}")
    max_depth = 8  # avoid overfitting
    reg = RandomForestRegressor(max_depth=max_depth).fit(X_train, y_train)
    y_pred_train = reg.predict(X_train)
    y_pred_test = reg.predict(X_test)
    rmse_train = round(compute_rmse(y_train, y_pred_train), 3)
    rmse_test = round(compute_rmse(y_test, y_pred_test), 3)
    print(f"Train rmse: {rmse_train}")
    print(f"Test rmse: {rmse_test}")


distances = np.array(distances).reshape(-1, 1)
X_train = distances[train_indices]
X_test = distances[test_indices]

fit_and_evaluate(X_train, y_train, X_test, y_test)

Start fitting
X_train shape: (489158, 1); X_test shape: (489158, 1)
Train rmse: 358.679
Test rmse: 360.239


### Feature engineering 1: Add node degrees
***
Based on our experience, the traffic is worse in the area with denser roads, downtown for exapmle. Therefore, we hypothesize that degrees of the two nodes of an edge can be helpful to predict the travelling time. As we can see from the result, the performance does significantly improve with these features.

In [7]:
degrees = np.array([[gcc.degree[s], gcc.degree[t]] for s, t in edges])

X_train = np.concatenate([X_train, degrees[train_indices]], axis=1)
X_test = np.concatenate([X_test, degrees[test_indices]], axis=1)

fit_and_evaluate(X_train, y_train, X_test, y_test)

Start fitting
X_train shape: (489158, 3); X_test shape: (489158, 3)
Train rmse: 337.259
Test rmse: 339.028


### Feature engineering 2: Add node coordinates
***
We also observe that traffic jam usually happens in some specific area such as freeway 405 near LAX airport. Hence, we add the coordinates of the two nodes of an edge as features and find that the performance further improves quite a lot.

In [8]:
coordinates = []
for s, t in edges:
    x1, y1 = loc_data[gcc.nodes[s]["name"]]["mean_coords"]
    x2, y2 = loc_data[gcc.nodes[t]["name"]]["mean_coords"]
    coordinates.append([x1, y1, x2, y2])
coordinates = np.array(coordinates)

X_train = np.concatenate([X_train, coordinates[train_indices]], axis=1)
X_test = np.concatenate([X_test, coordinates[test_indices]], axis=1)

fit_and_evaluate(X_train, y_train, X_test, y_test)

Start fitting
X_train shape: (489158, 7); X_test shape: (489158, 7)
Train rmse: 271.946
Test rmse: 274.231


### Feature engineering 3: Add eigenvector centrality
***
In addition to the degree of a node, we want to see if there are other node features that contain additional information of the travelling time. In this section, we add eigenvector centrality that measures the transitive influence of nodes. Relationships originating from high-scoring nodes contribute more to the score of a node than connections from low-scoring nodes. A high eigenvector score means that a node is connected to many nodes who themselves have high scores. However, it turns out that there is no improvement after adding this feature.

In [9]:
# cannot use edge weight
centralities = nx.eigenvector_centrality(gcc, weight=None)
edge_centralities = np.array([[centralities[s], centralities[t]] for s, t in edges])

X_train = np.concatenate([X_train, edge_centralities[train_indices]], axis=1)
X_test = np.concatenate([X_test, edge_centralities[test_indices]], axis=1)

fit_and_evaluate(X_train, y_train, X_test, y_test)

Start fitting
X_train shape: (489158, 9); X_test shape: (489158, 9)
Train rmse: 271.902
Test rmse: 274.213


### Feature engineering 4: Add clustering coefficient
***
In this section, we include another node feature, clustering coefficient, which is a measure of degree to which nodes in a graph tend to cluster together. The values for the clustering coefficient range from 0 to 1. A node has a clustering coefficient of 1 when it forms a clique with its neighbors, while a node has a clustering coefficient of 0 when there are no edges among the node's neighbors. With this feature added, the performance slightly improves.

In [10]:
# cannot use edge weight
clustering_coefs = nx.clustering(gcc, weight=None)
egde_clustering_coefs = np.array(
    [[clustering_coefs[s], clustering_coefs[t]] for s, t in edges]
)

X_train = np.concatenate([X_train, egde_clustering_coefs[train_indices]], axis=1)
X_test = np.concatenate([X_test, egde_clustering_coefs[test_indices]], axis=1)

fit_and_evaluate(X_train, y_train, X_test, y_test)

Start fitting
X_train shape: (489158, 11); X_test shape: (489158, 11)
Train rmse: 268.87
Test rmse: 271.124


### Correlation analysis
***
In this section, we show the correlation between the features we generate. As can be seen in table, the centrality features are extremely correlated with the degree features. It explains why there is no performance gain after adding the centrality features. Moreover, the high absolute correlation cooefficients between the clustering and the degree features are also the reason why there is only a little improvement with the clustering features included.

In [11]:
columns = [
    "length",
    "degrees_n1",
    "degrees_n2",
    "x_n1",
    "y_n1",
    "x_n2",
    "y_n2",
    "centrality_n1",
    "centrality_n2",
    "clustering_n1",
    "clustering_n2",
]
X_train_df = pd.DataFrame(X_train, columns=columns)
X_train_df.corr().style.background_gradient(axis=1)

,length,degrees_n1,degrees_n2,x_n1,y_n1,x_n2,y_n2,centrality_n1,centrality_n2,clustering_n1,clustering_n2
length,1.000000,0.268384,0.151738,0.084722,-0.006524,-0.006690,0.033429,0.256115,0.125316,-0.311586,-0.246348
degrees_n1,0.268384,1.000000,0.043623,-0.158522,-0.159184,0.002848,-0.041649,0.968324,0.058082,-0.929204,-0.034998
degrees_n2,0.151738,0.043623,1.000000,0.003394,0.172889,-0.211172,0.025246,0.096007,0.973098,0.006696,-0.899568
x_n1,0.084722,-0.158522,0.003394,1.000000,-0.294493,0.487253,-0.181675,-0.257197,-0.043235,-0.072529,-0.139475
y_n1,-0.006524,-0.159184,0.172889,-0.294493,1.000000,-0.226717,0.519351,-0.083452,0.180177,0.251179,-0.090883
x_n2,-0.006690,0.002848,-0.211172,0.487253,-0.226717,1.000000,-0.383525,-0.081503,-0.297082,-0.123339,-0.080059
y_n2,0.033429,-0.041649,0.025246,-0.181675,0.519351,-0.383525,1.000000,-0.003464,0.068735,0.094971,0.105507
centrality_n1,0.256115,0.968324,0.096007,-0.257197,-0.083452,-0.081503,-0.003464,1.000000,0.126640,-0.852970,-0.051968
centrality_n2,0.125316,0.058082,0.973098,-0.043235,0.180177,-0.297082,0.068735,0.126640,1.000000,0.013005,-0.810592
clustering_n1,-0.311586,-0.929204,0.006696,-0.072529,0.251179,-0.123339,0.094971,-0.852970,0.013005,1.000000,0.037197


### Feature engineering 5: Add SVD node embeddings
***
Finally, based on the experience from project 2, we guess that the graph-level features can be helpful for prediction. Therefore, we compute the graph-level node embeddings by utilizing the adjacency matrix of the road network to represent nodes. Since the dimension is too high, we reduce it by SVD. To generate edge features, we apply the method utilized by Sentence Transformer to capture the relation between the two nodes of an edge so as to represent an edge. As can be seen from the result, the model does have a significant performance gain.

In [12]:
# cannot use edge weight
adjacency_matrix = nx.adjacency_matrix(gcc, weight=None)
node_embeddings = TruncatedSVD(n_components=32).fit_transform(adjacency_matrix)
node_embeddings.shape

(2651, 32)

In [13]:
def get_edge_embeddings(indices):
    results = []
    for i in tqdm(indices):
        e1 = node_embeddings[edges[i][0]]
        e2 = node_embeddings[edges[i][1]]
        results.append(np.concatenate([e1, e2, e1 * e2, abs(e1 - e2)]))
    return np.array(results)


edge_embeddings_train = get_edge_embeddings(train_indices)
edge_embeddings_test = get_edge_embeddings(test_indices)

X_train_merge = np.concatenate([edge_embeddings_train, X_train], axis=1)
X_test_merge = np.concatenate([edge_embeddings_test, X_test], axis=1)

fit_and_evaluate(X_train_merge, y_train, X_test_merge, y_test)

  0%|          | 0/489158 [00:00<?, ?it/s]

  0%|          | 0/489158 [00:00<?, ?it/s]

Start fitting
X_train shape: (489158, 139); X_test shape: (489158, 139)
Train rmse: 254.304
Test rmse: 257.426


### Conclusions
***
In this mini project, we aim to predict the travelling time by generating several road (edge) features. With the length of the road as the only feature, a tree-based regressor can reduce 60% of RMSE from a random predictor. Furthermore, there is another 29% improvement after our feature engineering.

### Future work
***
By the experience from project 2, we believe that other graph-level node representations like Node2Vec or even GCN models can be more powerful. However, it would be very time-consuming to run those models on a super large network containing million of edges. Due to the limited time, we leave them to be our future work.